In [1]:
import scipy.io
import numpy as np
from scopt import scopt
import portfolio as pr
from frank_wolfe import frank_wolfe

In [2]:
problem_name = 'syn_1000_800_10_50'
data = scipy.io.loadmat('../../data/' + problem_name + '.mat')
R = data['W']
results = {problem_name: {}}

## Data preprocessing and parameters initialization

In [3]:
N, n = R.shape
Mf = 2
nu = 3
sigma_f = 10
diam_X=np.sqrt(2)
rho=2

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-10

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [4]:
func_x = lambda x: pr.portfolio(R,x)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_orcale = lambda grad: pr.linear_oracle_simplex(grad)
prox_func = lambda x, L: pr.proj_simplex(x)

## Run FW

In [5]:
#run_alpha_policies = ["standard","line_search","icml"]
run_alpha_policies = ['lloo']


for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_orcale,                                                    
                       x0,
                       FW_params,
                       hess=hess_x,
                       sigma_f=sigma_f,
                       diam_X=diam_X,
                       rho=rho,                                               
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=10, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
        'grad_hist': grad_hist
    }

********* Algorithm starts *********
Convergence achieved!
iter = 1, stepsize = 1.2717331468757236e-12, crit = 1.270938071052145e-12, upper_bound=0.05635009446772449, lower_bound=-5.176361103704231


In [6]:
alpha_hist

[8.228559592572766e-11]

## Run SCOPT

In [8]:
 x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
          grad_x,
          hess_mult_x,
          hess_mult_vec_x,
          Mf,
          nu,
          prox_func,
          x0,  
          sc_params,                                              
          eps=terminate_tol,                                              
          print_every=10)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    #'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

Fista err = 9.987e-06; Subiter = 444; subproblem converged!

iter =    1, stepsize = 5.670e-01, rdiff = 4.783e-01 , f = 0.0212893

Fista err = 8.721e-06; Subiter =  16; subproblem converged!

Fista err = 1.583e-06; Subiter =   2; subproblem converged!

Fista err = 6.048e-07; Subiter =   2; subproblem converged!

Fista err = 7.147e-07; Subiter =   2; subproblem converged!

Convergence achieved!
iter =    7, stepsize = 0.000e+00, rdiff = 0.000e+00,value=-4.47509

17.13429880142212
